### Import Lib

In [8]:
import os
import sys
import math
import yaml
import torch
import joblib
import warnings
import torch.nn as nn

### Utility files

In [9]:
def config_files():
    with open("../config.yml", "r") as config_file:
        return yaml.safe_load(config_file)


def dump_file(value: None, filename: None):
    if (value is not None) and (filename is not None):
        joblib.dump(value=value, filename=filename)
    else:
        print("Error: 'value' and 'filename' must be provided.".capitalize())


def load_file(filename: None):
    if filename is not None:
        return joblib.load(filename=filename)
    else:
        print("Error: 'filename' must be provided.".capitalize())


def weight_init(m):
    classname = m.__class__.__name__
    if classname.find("Linear") != -1:
        nn.init.kaiming_normal_(m.weight.data)
        if m.bias is not None:
            nn.init.constant_(m.bias.data, 0.0)
    elif classname.find("Conv") != -1:
        nn.init.kaiming_normal_(m.weight.data)
        if m.bias is not None:
            nn.init.constant_(m.bias.data, 0.0)


def device_init(device: str = "cuda"):
    if device == "cuda":
        return torch.device("cuda" if torch.cuda.is_available() else "cpu")
    elif device == "mps":
        return torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    else:
        return torch.device("cpu")

### Patch Embedding

In [10]:
class PatchEmbedding(nn.Module):
    def __init__(
        self,
        channels: int = 3,
        patch_size: int = 16,
        image_size: int = 128,
        dimension: int = None,
    ):
        super(PatchEmbedding, self).__init__()

        self.in_channels = channels
        self.patch_size = patch_size
        self.dimension = dimension
        self.image_size = image_size

        if self.dimension is None:
            self.dimension = (self.patch_size**2) * self.in_channels

        self.number_of_pathches = (self.image_size // self.patch_size) ** 2

        self.kernel_size, self.stride_size = self.patch_size, self.patch_size
        self.padding_size = self.patch_size // self.patch_size

        self.projection = nn.Conv2d(
            in_channels=self.in_channels,
            out_channels=self.dimension,
            kernel_size=self.kernel_size,
            stride=self.stride_size,
            padding=self.padding_size,
            bias=False,
        )

        self.positional_embeddings = nn.Parameter(
            torch.randn(self.padding_size, self.number_of_pathches, self.dimension),
            requires_grad=True,
        )

    def forward(self, x: torch.Tensor):
        if isinstance(x, torch.Tensor):
            x = self.projection(x)
            x = x.view(x.size(0), x.size(1), -1)
            x = x.permute(0, 2, 1)
            x = self.positional_embeddings + x
            return x
        else:
            raise ValueError("Input must be a torch tensor.".capitalize())

    @staticmethod
    def total_params(model):
        if isinstance(model, PatchEmbedding):
            return sum(params.numel() for params in model.parameters())


if __name__ == "__main__":
    image_channels = config_files()["patchEmbeddings"]["channels"]
    image_size = config_files()["patchEmbeddings"]["image_size"]
    patch_size = config_files()["patchEmbeddings"]["patch_size"]
    dimension = config_files()["patchEmbeddings"]["dimension"]

    pathEmbedding = PatchEmbedding(
        channels=image_channels,
        patch_size=patch_size,
        image_size=image_size,
        dimension=dimension,
    )

    image = torch.randn(
        (image_channels // image_channels, image_channels, image_size, image_size)
    )

    assert (pathEmbedding(image).size()) == (
        image_channels // image_channels,
        (image_size // patch_size) ** 2,
        dimension,
    )

### Scaled Dot Product -> self attention 

In [11]:
def scaled_dot_product(query: torch.Tensor, key: torch.Tensor, values: torch.Tensor):
    if (
        isinstance(query, torch.Tensor)
        and isinstance(key, torch.Tensor)
        and isinstance(values, torch.Tensor)
    ):
        logits = torch.matmul(query, key.transpose(-1, -2)) / math.sqrt(query.size(-1))
        attention_weights = torch.softmax(logits, dim=-1)
        attention_output = torch.matmul(attention_weights, values)
        return attention_output

    else:
        raise ValueError("All inputs must be torch tensors.".capitalize())

if __name__ == "__main__":
    query = torch.randn((1, 8, 64, 32))
    key = torch.randn((1, 8, 64, 32))
    values = torch.randn((1, 8, 64, 32))

    assert (scaled_dot_product(query, key, values).size()) == (1, 8, 64, 32)

### Transfomer Encoder Block -> attention is all you need

In [15]:
class TransformerEncoderBlock(nn.Module):
    def __init__(self, nheads: int = 8, dimension: int = 256):
        super(TransformerEncoderBlock, self).__init__()
        self.nheads = nheads
        self.dimension = dimension

        warnings.warn(
            "Please ensure that the dimension is a multiple of the number of heads in the encoder block (e.g., 256 % 8 = 0). "
            "This is a requirement for the Transformer Encoder Block to function properly. "
            "If not, you might need to adjust the dimension or the number of heads."
        )
        assert (
            dimension % self.nheads == 0
        ), "Dimension mismatched with nheads and dimension".title()

        self.QKV = nn.Linear(
            in_features=self.dimension, out_features=3 * self.dimension, bias=False
        )

    def forward(self, x: torch.Tensor):
        if isinstance(x, torch.Tensor):
            QKV = self.QKV(x)
            query, key, values = torch.chunk(input=QKV, chunks=3, dim=-1)
            query = query.view(
                query.size(0), query.size(1), self.nheads, self.dimension // self.nheads
            )
            key = key.view(
                key.size(0), key.size(1), self.nheads, self.dimension // self.nheads
            )
            values = values.view(
                values.size(0),
                values.size(1),
                self.nheads,
                self.dimension // self.nheads,
            )

            query = query.permute(0, 2, 1, 3)
            key = key.permute(0, 2, 1, 3)
            values = values.permute(0, 2, 1, 3)

            attention_output = scaled_dot_product(query=query, key=key, values=values)
            attention_output = attention_output.view(
                attention_output.size(0),
                attention_output.size(2),
                attention_output.size(1) * attention_output.size(-1),
            )

            return attention_output
        else:
            raise ValueError("Input must be a torch tensor.".capitalize())

    @staticmethod
    def total_params(model):
        if isinstance(model, TransformerEncoderBlock):
            return sum(params.numel() for params in model.parameters())

if __name__ == "__main__":
    nheads = config_files()["transfomerEncoderBlock"]["nheads"]
    dimension = config_files()["patchEmbeddings"]["dimension"]
    image_channels = config_files()["patchEmbeddings"]["channels"]
    image_size = config_files()["patchEmbeddings"]["image_size"]
    patch_size = config_files()["patchEmbeddings"]["patch_size"]
    dimension = config_files()["patchEmbeddings"]["dimension"]

    transformerEncoderBlock = TransformerEncoderBlock(nheads=nheads, dimension=dimension)

    image = torch.randn((1, (image_size // patch_size) ** 2, dimension))

    assert (transformerEncoderBlock(image).size()) == (
        (1, (image_size // patch_size) ** 2, dimension)
    )

/var/folders/_6/fpk79cm53rxgcj2gh5prtww00000gn/T/ipykernel_38456/2270339858.py:7: UserWarning: Please ensure that the dimension is a multiple of the number of heads in the encoder block (e.g., 256 % 8 = 0). This is a requirement for the Transformer Encoder Block to function properly. If not, you might need to adjust the dimension or the number of heads.
  warnings.warn(
